<a href="https://colab.research.google.com/github/Euan-Kearney/cancer-cam/blob/main/model_0_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
pip install -q opendatasets
pip install -q pandas
pip install -q kaggle

In [2]:
from google.colab import userdata
import json
import os

kaggle_json_str = userdata.get('KAGGLE_JSON')
kaggle_dict = json.loads(kaggle_json_str)
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_dict, f)

!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 -p /content/HAM10000 --unzip --quiet

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0


In [4]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
import shutil

base = 'HAM10000_organised'
splits = ['train', 'validate', 'test']
os.makedirs(base, exist_ok=True)

classes = [
    'melanoma',
    'BCC',
    'SCC',
    'precancerous',
    'low_risk',
    'unknown'
]

for split in splits:
    for cls in classes:
        path = os.path.join(base, split, cls)
        os.makedirs(path, exist_ok=True)

os.makedirs(base, exist_ok=True)

def map_classes(cls):
  if cls == 'mel':
    return 'melanoma'
  elif cls == 'bcc':
    return 'BCC'
  elif cls == 'akiec':
    return 'precancerous'
  elif cls == 'bkl':
    return 'low_risk'
  elif cls == 'df':
    return 'low_risk'
  elif cls == 'nv':
    return 'low_risk'
  elif cls == 'vasc':
    return 'low_risk'

def copy_images(split, dataframe):
  for index, row in dataframe.iterrows():
    image_name = row['image_id'] + '.jpg'
    cls = map_classes(row['dx'])
    part_1_source = os.path.join('HAM10000/HAM10000_images_part_1/', image_name)
    part_2_source = os.path.join('HAM10000/HAM10000_images_part_2/', image_name)
    destination = os.path.join('HAM10000_organised/', split, cls, image_name)
    if os.path.exists(destination):
      break
    if os.path.exists(part_1_source):
      shutil.copyfile(part_1_source, destination)
    elif os.path.exists(part_2_source):
      shutil.copyfile(part_2_source, destination)

all_data = pd.read_csv('HAM10000/HAM10000_metadata.csv')
train_df, temp_df = train_test_split(all_data, test_size=0.2)
validate_df, test_df = train_test_split(temp_df, test_size=0.5)

for split in splits:
  if split == 'train':
    copy_images(split, train_df)
  elif split == 'validate':
    copy_images(split, validate_df)
  else:
    copy_images(split, test_df)








